# Dataset 1
## Importing the data

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import statistics as st
from sklearn import neighbors

In [2]:
# Adult dataset
header = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
       'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']

#convert 
missing_values = [' ?', '?']
data_df = pd.read_csv('C:/Jiayang/JupyterLab/Comp_551/mini_project_1/adult.data',header=None,na_values = missing_values,names = header)
test_df = pd.read_csv('C:/Jiayang/JupyterLab/Comp_551/mini_project_1/adult.test',header = None,na_values = missing_values,names = header)

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  31978 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [4]:
#number of rows x number of columns
print(data_df.shape)
data_df.head()

(32561, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
# Check all entries if there are invalid data
for col in data_df.columns:
  if data_df[col].dtype == "object":
    print('Attribute name:',col)
    print(data_df[col].value_counts())
    print('\n')

Attribute name: workclass
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64


Attribute name: education
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64


Attribute name: marital-status
 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital-status, dtype: int64


Attribut

# Data Cleaning

In [6]:
# Create a copy of the dataframe(checkpoint) before dropna
df = data_df.copy(deep = True)

In [7]:
# drop invalid data rows
# ? is already converted to Nan while reading_csv
df = df.dropna()
print(df.shape)

(30162, 15)


In [8]:
#check all is dropped
null_data = df.iloc[:,:-1]
null_data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64

# One-hot Encoding

In [9]:
# Create copies of dataset(checkpoint) before conveting 
df_copy = df.copy(deep = True)

In [10]:
# Converting 'sex' and 'salary' to numerical

# Male = 1, Female = 0
df_copy["sex"] = np.where(df.sex == ' Male', 1, 0)
# Salary > 50K = 1, < 50K = 0
df_copy["salary"] = np.where(df.salary == ' >50K', 1 , 0)


In [11]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [12]:
# Check object data type
obj_col= []
for col in df_copy.columns:
    if df_copy[col].dtype == 'object':
        obj_col.append(col)
print(obj_col)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']


In [13]:
# Check int data type
int_col= []
for col in df_copy.columns:
    if df_copy[col].dtype == 'int64' or df_copy[col].dtype == 'int32':
        int_col.append(col)
print(int_col)

['age', 'fnlwgt', 'education-num', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'salary']


In [14]:
from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()
std_scaler

# Normalize numeric attributes and assign it into a new dataframe
df_int = pd.DataFrame(std_scaler.fit_transform(df_copy[int_col]), columns=int_col)

In [15]:
df_int.head()

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,salary
0,0.042796,-1.062722,1.128918,0.692806,0.146092,-0.218586,-0.077734,-0.575691
1,0.880288,-1.007871,1.128918,0.692806,-0.147445,-0.218586,-2.331531,-0.575691
2,-0.033340,0.244693,-0.439738,0.692806,-0.147445,-0.218586,-0.077734,-0.575691
3,1.108695,0.425240,-1.224066,0.692806,-0.147445,-0.218586,-0.077734,-0.575691
4,-0.794697,1.406658,1.128918,-1.443405,-0.147445,-0.218586,-0.077734,-0.575691


In [31]:
#one hot encoding 

workclass = pd.get_dummies(df_copy['workclass'])
education = pd.get_dummies(df_copy['education'])
marital_status = pd.get_dummies(df_copy['marital-status'])
occupation = pd.get_dummies(df_copy['occupation'])
relationship = pd.get_dummies(df_copy['relationship'])
race = pd.get_dummies(df_copy['race'])
native_country = pd.get_dummies(df_copy['native-country'])

df_obj = pd.concat([workclass,
                    education,
                    marital_status,
                    occupation,
                    relationship,
                    race,
                    native_country
                   ], axis=1)

In [33]:
df_obj.head()

,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,10th,11th,12th,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# from sklearn.preprocessing import OrdinalEncoder
# ord_enc = OrdinalEncoder()

# # enconde categorical features into numerical features
# df_obj = df.copy()
# df_obj.drop(int_col,axis='columns', inplace=True)

# #print(df_obj)

# for col in df_obj.columns[:]:
#   if df_obj[col].dtype == "object":
#     df_obj[col] = ord_enc.fit_transform(df_obj[[col]])
    
# print(df_obj)

       workclass  education  marital-status  occupation  relationship  race  \
0            5.0        9.0             4.0         0.0           1.0   4.0   
1            4.0        9.0             2.0         3.0           0.0   4.0   
2            2.0       11.0             0.0         5.0           1.0   4.0   
3            2.0        1.0             2.0         5.0           0.0   2.0   
4            2.0        9.0             2.0         9.0           5.0   2.0   
...          ...        ...             ...         ...           ...   ...   
32556        2.0        7.0             2.0        12.0           5.0   4.0   
32557        2.0       11.0             2.0         6.0           0.0   4.0   
32558        2.0       11.0             6.0         0.0           4.0   4.0   
32559        2.0       11.0             4.0         0.0           3.0   4.0   
32560        3.0       11.0             2.0         3.0           5.0   4.0   

       native-country  
0                38.0  
1  

In [36]:
# Insert id column to different datasets
df_int.insert(0, 'id', range(0, 0 + len(df_int)))
df_obj.insert(0, 'id', range(0, 0 + len(df_obj)))

ValueError: cannot insert id, already exists

In [37]:
# Merge all datasets
df_feature = pd.merge(df_int,df_obj, on=["id"])

# Show the the tope 5 obersrvation of the dataset
df_feature.head(5)

,id,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,Federal-gov,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,0,0.042796,-1.062722,1.128918,0.692806,0.146092,-0.218586,-0.077734,-0.575691,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0.880288,-1.007871,1.128918,0.692806,-0.147445,-0.218586,-2.331531,-0.575691,0,...,0,0,0,0,0,0,0,1,0,0
2,2,-0.033340,0.244693,-0.439738,0.692806,-0.147445,-0.218586,-0.077734,-0.575691,0,...,0,0,0,0,0,0,0,1,0,0
3,3,1.108695,0.425240,-1.224066,0.692806,-0.147445,-0.218586,-0.077734,-0.575691,0,...,0,0,0,0,0,0,0,1,0,0
4,4,-0.794697,1.406658,1.128918,-1.443405,-0.147445,-0.218586,-0.077734,-0.575691,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#drop id column
df_feature.drop('id',axis='columns', inplace=True)

In [39]:
df_feature.head(5)

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,salary,Federal-gov,Local-gov,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,0.042796,-1.062722,1.128918,0.692806,0.146092,-0.218586,-0.077734,-0.575691,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.880288,-1.007871,1.128918,0.692806,-0.147445,-0.218586,-2.331531,-0.575691,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.033340,0.244693,-0.439738,0.692806,-0.147445,-0.218586,-0.077734,-0.575691,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1.108695,0.425240,-1.224066,0.692806,-0.147445,-0.218586,-0.077734,-0.575691,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-0.794697,1.406658,1.128918,-1.443405,-0.147445,-0.218586,-0.077734,-0.575691,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
data = df_feature.to_numpy()
data.dtype

dtype('float64')

# KNN K-fold Cross Validation

In [41]:
def split(dataset, folds):
	
	return np.array_split(dataset, folds)

In [53]:
def KNN_cross_validation(dataset, folds, K):
    
    best_acc = 0
    best_config = 0
    best_model = neighbors.KNeighborsClassifier(n_neighbors=1)
    
    for k in np.arange(1,K):

        model = neighbors.KNeighborsClassifier(n_neighbors=k)
        
        data_splited = split(dataset, folds)
        
        for i in range(folds):
            
            acc_l = list()
            validation = data_splited[i]

            r = list(range(folds))
            r.pop(i)

            for j in r:

                if j == r[0] :
                    training = data_splited[j]
                else:
                    training = np.concatenate((training,data_splited[j]), axis=0)

            # model.fit(training[:,:14], training[:,14])
            # pred = model.predict(validation[:,:14])
            # acc = np.sum(pred == validation[:,14])/len(validation[:,14])
            # acc_l += acc

            model.fit(training[:,:-1],training[:,-1])
            pred = model.predict(validation[:,:-1])
            acc = np.mean(pred == validation[:,-1])
            acc_l.append(acc)

        acc_m = st.mean(acc_l)
        
        if best_acc < acc_m:
            best_acc = acc_m
            best_config = k
            best_model = model

    return  best_acc, best_config, best_model

# Decision Tree K-fold Cross Validation

In [54]:
from sklearn.tree import DecisionTreeClassifier

def DT_cross_validation(dataset, folds, depth):
    
    best_acc = 0
    best_config = 0
    best_model = DecisionTreeClassifier(max_depth=1)

    for d in np.arange(1,depth):

        model = DecisionTreeClassifier(max_depth=d)
        
        data_splited = split(dataset, folds)
        
        for i in range(folds):
            
            acc_l = list()
            validation = data_splited[i]

            r = list(range(folds))
            r.pop(i)

            for j in r:

                if j == r[0] :
                    training = data_splited[j]
                else:
                    training = np.concatenate((training,data_splited[j]), axis=0)
        
            model.fit(training[:,:-1],training[:,-1])
            pred = model.predict(validation[:,:-1])
            acc = np.mean(pred == validation[:,-1])
            acc_l.append(acc)

        acc_m = st.mean(acc_l)

        if best_acc < acc_m:
                best_acc = acc_m
                best_config = d
                best_model = model
    
    return  best_acc, best_config, best_model


# Trainning - Dataset 1

In [55]:
size = data.shape[0]

size20 = int(size * 0.2)

size40 = int(size * 0.4)

size60 = int(size * 0.6)

size80 = int(size * 0.8)

In [56]:
acc1, config1, KNN_model1 = KNN_cross_validation(data[:size20], 5, 20)
print(acc1, config1, KNN_model1)

0.9991708126036484 2 KNeighborsClassifier(n_neighbors=2)


In [58]:
pred = KNN_model1.predict(data[size20:,:-1])
acc = np.mean(pred == data[size20:,-1])
print(acc)

0.999461251554082


In [ ]:
acc2, config2, KNN_model2 = KNN_cross_validation(data[:size40], 5, 20)
print(acc2, config2, KNN_model2)

In [ ]:
pred = KNN_model2.predict(data[size40:,:14])
acc = np.mean(pred == data[size40:,14])
print(acc)

In [ ]:
acc3, config3, KNN_model3 = KNN_cross_validation(data[:size60], 5, 20)
print(acc3, config3, KNN_model3)

In [ ]:
pred = KNN_model3.predict(data[size60:,:14])
acc = np.mean(pred == data[size60:,14])
print(acc)

In [67]:
%%time
acc4, config4, KNN_model4 = KNN_cross_validation(data[:size80], 5, 20)
print(acc4, config4, KNN_model4)

0.9995854922279793 1 KNeighborsClassifier(n_neighbors=1)


In [69]:
pred = KNN_model4.predict(data[size80:,:-1])
acc = np.mean(pred == data[size80:,-1])
print(acc)

0.9988397149013758


In [66]:
dtacc1, dtconfig1, dtmodel1 = DT_cross_validation(data[:size20], 5, 15)
print(dtacc1, dtconfig1, dtmodel1)

0.9991708126036484 1 DecisionTreeClassifier(max_depth=1)


In [ ]:
pred = dtmodel1.predict(data[size20:,:14])
acc = np.mean(pred == data[size20:,14])
print(acc)

In [ ]:
dtacc2, dtconfig2, dtmodel2 = DT_cross_validation(data[:size40], 5, 15)
print(dtacc2, dtconfig2, dtmodel2)

In [ ]:
pred = dtmodel2.predict(data[size40:,:14])
acc = np.mean(pred == data[size40:,14])
print(acc)

In [ ]:
dtacc3, dtconfig3, dtmodel3 = DT_cross_validation(data[:size60], 5, 15)
print(dtacc3, dtconfig3, dtmodel3)

In [ ]:
pred = dtmodel3.predict(data[size60:,:14])
acc = np.mean(pred == data[size60:,14])
print(acc)

In [ ]:
dtacc4, dtconfig4, dtmodel4 = DT_cross_validation(data[:size80], 5, 15)
print(dtacc4, dtconfig4, dtmodel4)

In [ ]:
pred = dtmodel4.predict(data[size80:,:14])
acc = np.mean(pred == data[size80:,14])
print(acc)

# Trainning - Dataset 2

In [71]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Load data
data = pd.read_csv('C:/Jiayang/JupyterLab/Comp_551/mini_project_1/bank-additional-full.csv', sep = ";")
# Variables names
var_names = data.columns.tolist()

# Categorical vars
categs = ['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome','y']
# Quantitative vars
quantit = [i for i in var_names if i not in categs]

# Get dummy variables for categorical vars
job = pd.get_dummies(data['job'])
marital = pd.get_dummies(data['marital'])
education = pd.get_dummies(data['education'])
default = pd.get_dummies(data['default'])
housing = pd.get_dummies(data['housing'])
loan = pd.get_dummies(data['loan'])
contact = pd.get_dummies(data['contact'])
month = pd.get_dummies(data['month'])
day = pd.get_dummies(data['day_of_week'])
poutcome = pd.get_dummies(data['poutcome'])

# Map variable to predict
dict_map = dict()
y_map = {'yes':1,'no':0}
dict_map['y'] = y_map
data = data.replace(dict_map)
label = data['y']

df1 = data[quantit]
df1_names = df1.keys().tolist()

# Scale quantitative variables
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df1)
df1 = pd.DataFrame(x_scaled)
df1.columns = df1_names


# Get final df
final_df = pd.concat([df1,
                      job,
                      marital,
                      education,
                      default,
                      housing,
                      loan,
                      contact,
                      month,
                      day,
                      poutcome,
                      label], axis=1)

# Quick check
final_df.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,sep,fri,mon,thu,tue,wed,failure,nonexistent,success,y
0,0.481481,0.053070,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,1,0,0,0,0,1,0,0
1,0.493827,0.030297,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,1,0,0,0,0,1,0,0
2,0.246914,0.045954,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,1,0,0,0,0,1,0,0
3,0.283951,0.030704,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,1,0,0,0,0,1,0,0
4,0.481481,0.062424,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,0,1,0,0,0,0,1,0,0


In [73]:
data2 = final_df.to_numpy()
data2[0,55].dtype

dtype('float64')

In [74]:
size = data2.shape[0]

size20 = int(size * 0.2)

size40 = int(size * 0.4)

size60 = int(size * 0.6)

size80 = int(size * 0.8)

In [77]:
%%time

acc1, config1, KNN_model1 = KNN_cross_validation(data2[:size20], 5, 20)
print(acc1, config1, KNN_model1)

0.9587128111718276 2 KNeighborsClassifier(n_neighbors=2)
Wall time: 28.2 s


In [78]:
pred = KNN_model1.predict(data2[size20:,:-1])
acc = np.mean(pred == data2[size20:,-1])
print(acc)

0.8673484871475827
